In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
from pynq import Overlay
overlay = Overlay('./bitstream/DEC_rLUT4bits.bit')

In [3]:
overlay.ip_dict

{'DEC_rLUT4bits_0': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xaf8776d0>,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'DEC_rLUT4bits_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S00_AXI',
  'parameters': {'C_S00_AXI_ADDR_WIDTH': '4',
   'C_S00_AXI_BASEADDR': '0x43C00000',
   'C_S00_AXI_DATA_WIDTH': '32',
   'C_S00_AXI_HIGHADDR': '0x43C0FFFF',
   'Component_Name': 'DEC_rLUT4bits_DEC_rLUT4bits_0_0',
   'EDK_IPTYPE': 'PERIPHERAL'},
  'phys_addr': 1136656384,
  'registers': {},
  'state': None,
  'type': 'xilinx.com:user:DEC_rLUT4bits:1.0'}}

In [4]:
from pynq import MMIO
ip = MMIO(0x43C00000, 0x10000)

In [5]:
ip.device

### 將 r_LUT  的location輸出值轉為2補數:

In [6]:
def twos_complement_32bit(x):
    """將 32-bit 補數整數轉為有號值"""
    if x >= (1 << 31):
        return x - (1 << 32)
    else:
        return x

#### Test:

In [7]:
remainder_1 = 353

ip.write_mm(0, remainder_1)

location_1 = ip.read_mm(4)
location_2 = ip.read_mm(8)


print(twos_complement_32bit(location_1))
print(twos_complement_32bit(location_2))

-5
11


### 開始則中查表驗證

## 折衷查表:
### 用 Single Error Correction 解 Double Error Correction (A 取Double_AWE的 A)

#### 1.取 A的bits $\approx$ N的bits $\Rightarrow$ A = 18613  (30bits), N的範圍: 0 ~ 1073741823 (30bits)

### Define function
* (Not including 2's complement range

In [8]:
import math
import numpy as np


def decimal_to_binary(decimal_number):
    if decimal_number == 0:
        return "0"
    # 處理正數：直接轉換
    if decimal_number > 0:
        return bin(decimal_number)[2:]  # 移除 "0b" 前綴
       # 處理負數：自動計算所需位元數（2's complement）
    abs_value = abs(decimal_number)
    bits = abs_value.bit_length() + 1  # 負數多一個符號位元
    two_complement_value = (1 << bits) + decimal_number
    
    return bin(two_complement_value)[2:]  # 移除 "0b" 前綴

def bits_needed(number):
    return number.bit_length()


def bits_range(bits, signed=False):     ## 可選擇設定 有號數 或 無號數
    if bits <= 0:
        return "位元數必須為正整數！"
    
    if signed:
        # 二補數範圍: -2^(bits-1) 到 2^(bits-1)-1
        min_value = -(2**(bits - 1))
        max_value = 2**(bits - 1) - 1
    else:
        # 無符號範圍: 0 到 2^bits - 1
        min_value = 0
        max_value = 2**bits - 1

    return min_value, max_value

## 把重複的 AWE 去除
def remove_duplicates(input_list):
    unique_list = []
    for number in input_list:
        if number not in unique_list:
            unique_list.append(number)
    return unique_list


def get_sign(number):
    return 1 if number > 0 else -1 if number < 0 else 0


print("一個2補數所需要的位元數",bits_needed(120))
print("一bits數可表示的2補數的範圍",bits_range(bits_needed(120)))
print("一個2補數表達為二進制為:",decimal_to_binary(-8))

一個2補數所需要的位元數 7
一bits數可表示的2補數的範圍 (0, 127)
一個2補數表達為二進制為: 11000


In [10]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data1 = (1 << 4) - 1   ## 
data2 = (1 << 4) - 1
data3 = (1 << 4) - 2
AN = 665
input1 = data1*AN
input2 = data2*AN
input3 = data3*AN
bits1 = bits_needed(input1)
bits2 = bits_needed(input2)
bits3 = bits_needed(input3)
print("inputs1的位元數:", bits1)
print("inputs2的位元數:", bits2)
print("inputs3的位元數:", bits3)

inputs1的位元數: 14
inputs2的位元數: 14
inputs3的位元數: 14


### 產生1-bit AWE (Arithmetic-Weight Error)
* 並產生 Delta Q 的 list

In [11]:
AWE1_1=[]
e1_1=0     ## 錯誤數量
e1_data1=[]
h = [];   #位置
for i in range(0, bits1):
    e1_data1.insert(e1_1 , input1 + 2**i)
    AWE1_1.insert(e1_1, 2**i)
    e1_1 = e1_1 + 1
    e1_data1.insert(e1_1 , input1 - 2**i)
    AWE1_1.insert(e1_1, -2**i)
    e1_1 = e1_1 + 1
    h.insert(i,i+1)

AWE1_2=[]
e1_2=0     ## 錯誤數量
e1_data2=[]
for i in range(0, bits2):
    e1_data2.insert(e1_2 , input2 + 2**i)
    AWE1_2.insert(e1_2, 2**i)
    e1_2 = e1_2 + 1
    e1_data2.insert(e1_2 , input2 - 2**i)
    AWE1_2.insert(e1_2, -2**i)
    e1_2 = e1_2 + 1
    h.insert(i,i+1)
    
AWE1_3=[]
e1_3=0     ## 錯誤數量
e1_data3=[]
for i in range(0, bits3):
    e1_data3.insert(e1_3 , input3 + 2**i)
    AWE1_3.insert(e1_3, 2**i)
    e1_3 = e1_3 + 1
    e1_data3.insert(e1_3 , input3 - 2**i)
    AWE1_3.insert(e1_3, -2**i)
    e1_3 = e1_3 + 1
    h.insert(i,i+1)

h_awe = [val for num in h for val in (num, -num)]
print("產生1-bit AWE:", AWE1_1)
print("不同輸入資料的AWE是否相等:",AWE1_1 == AWE1_2 == AWE1_3)
print(h_awe)
print(np.size(AWE1_1))

產生1-bit AWE: [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192]
不同輸入資料的AWE是否相等: True
[1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13, -13, 14, -14, 1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13, -13, 14, -14, 1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13, -13, 14, -14]
28


### 產生SEC的模數:

In [12]:
SEC_R = []
AWE1_R= []
for m in range(0,e1_1):
    mod = e1_data1[m] % AN
    SEC_R.append(mod)

for n in range(0,e1_1):
    mod_awe = AWE1_3[n] % AN
    AWE1_R.append(mod_awe)


print("SEC的模數:\n",SEC_R)
print("AWE1的模數:\n",SEC_R)
print("AWE1的模數是否等於SEC模數:", SEC_R == AWE1_R)
print("SEC的大小:",np.size(SEC_R))

SEC的模數:
 [1, 664, 2, 663, 4, 661, 8, 657, 16, 649, 32, 633, 64, 601, 128, 537, 256, 409, 512, 153, 359, 306, 53, 612, 106, 559, 212, 453]
AWE1的模數:
 [1, 664, 2, 663, 4, 661, 8, 657, 16, 649, 32, 633, 64, 601, 128, 537, 256, 409, 512, 153, 359, 306, 53, 612, 106, 559, 212, 453]
AWE1的模數是否等於SEC模數: True
SEC的大小: 28


### 產生2-bit AWE (Arithmetic-Weight Error)

In [13]:
AWE_2=[]
e2=0     ## 錯誤數量
p=0      ## 錯誤位置
e2_data=[]
for i in range(0, bits1):
    for j in range(i+1, bits1):
        e2_data.insert(e2 , (input1 + 2**i + 2**j))
        AWE_2.insert(e2 , (2**i + 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 + 2**i - 2**j))
        AWE_2.insert(e2 , (2**i - 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i + 2**j))
        AWE_2.insert(e2 , (-2**i + 2**j))                
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i -2**j))
        AWE_2.insert(e2 , (-2**i - 2**j))
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        p = p + 1


print(e2)
print(p)
print("產生2-bits AWE:",AWE_2)
print("產生2-bits 錯誤資料:",e2_data)

第 0 筆錯誤資料: 9978
第 0 筆算術錯誤: 3
第 1 筆錯誤資料: 9974
第 1 筆算術錯誤: -1
第 2 筆錯誤資料: 9976
第 2 筆算術錯誤: 1
第 3 筆錯誤資料: 9972
第 3 筆算術錯誤: -3
第 4 筆錯誤資料: 9980
第 4 筆算術錯誤: 5
第 5 筆錯誤資料: 9972
第 5 筆算術錯誤: -3
第 6 筆錯誤資料: 9978
第 6 筆算術錯誤: 3
第 7 筆錯誤資料: 9970
第 7 筆算術錯誤: -5
第 8 筆錯誤資料: 9984
第 8 筆算術錯誤: 9
第 9 筆錯誤資料: 9968
第 9 筆算術錯誤: -7
第 10 筆錯誤資料: 9982
第 10 筆算術錯誤: 7
第 11 筆錯誤資料: 9966
第 11 筆算術錯誤: -9
第 12 筆錯誤資料: 9992
第 12 筆算術錯誤: 17
第 13 筆錯誤資料: 9960
第 13 筆算術錯誤: -15
第 14 筆錯誤資料: 9990
第 14 筆算術錯誤: 15
第 15 筆錯誤資料: 9958
第 15 筆算術錯誤: -17
第 16 筆錯誤資料: 10008
第 16 筆算術錯誤: 33
第 17 筆錯誤資料: 9944
第 17 筆算術錯誤: -31
第 18 筆錯誤資料: 10006
第 18 筆算術錯誤: 31
第 19 筆錯誤資料: 9942
第 19 筆算術錯誤: -33
第 20 筆錯誤資料: 10040
第 20 筆算術錯誤: 65
第 21 筆錯誤資料: 9912
第 21 筆算術錯誤: -63
第 22 筆錯誤資料: 10038
第 22 筆算術錯誤: 63
第 23 筆錯誤資料: 9910
第 23 筆算術錯誤: -65
第 24 筆錯誤資料: 10104
第 24 筆算術錯誤: 129
第 25 筆錯誤資料: 9848
第 25 筆算術錯誤: -127
第 26 筆錯誤資料: 10102
第 26 筆算術錯誤: 127
第 27 筆錯誤資料: 9846
第 27 筆算術錯誤: -129
第 28 筆錯誤資料: 10232
第 28 筆算術錯誤: 257
第 29 筆錯誤資料: 9720
第 29 筆算術錯誤: -255
第 30 筆錯誤資料: 10230
第 30 筆算術錯誤: 255
第 31 筆錯誤資料

91
產生2-bits AWE: [3, -1, 1, -3, 5, -3, 3, -5, 9, -7, 7, -9, 17, -15, 15, -17, 33, -31, 31, -33, 65, -63, 63, -65, 129, -127, 127, -129, 257, -255, 255, -257, 513, -511, 511, -513, 1025, -1023, 1023, -1025, 2049, -2047, 2047, -2049, 4097, -4095, 4095, -4097, 8193, -8191, 8191, -8193, 6, -2, 2, -6, 10, -6, 6, -10, 18, -14, 14, -18, 34, -30, 30, -34, 66, -62, 62, -66, 130, -126, 126, -130, 258, -254, 254, -258, 514, -510, 510, -514, 1026, -1022, 1022, -1026, 2050, -2046, 2046, -2050, 4098, -4094, 4094, -4098, 8194, -8190, 8190, -8194, 12, -4, 4, -12, 20, -12, 12, -20, 36, -28, 28, -36, 68, -60, 60, -68, 132, -124, 124, -132, 260, -252, 252, -260, 516, -508, 508, -516, 1028, -1020, 1020, -1028, 2052, -2044, 2044, -2052, 4100, -4092, 4092, -4100, 8196, -8188, 8188, -8196, 24, -8, 8, -24, 40, -24, 24, -40, 72, -56, 56, -72, 136, -120, 120, -136, 264, -248, 248, -264, 520, -504, 504, -520, 1032, -1016, 1016, -1032, 2056, -2040, 2040, -2056, 4104, -4088, 4088, -4104, 8200, -8184, 8184, -8200, 

In [14]:
DEC_R = []
for m in range(0,e2):
    mod = e2_data[m] % AN
    DEC_R.append(mod)

print("DEC_R的模數:\n",DEC_R)
print(np.size(DEC_R))

DEC_R的模數:
 [3, 664, 1, 662, 5, 662, 3, 660, 9, 658, 7, 656, 17, 650, 15, 648, 33, 634, 31, 632, 65, 602, 63, 600, 129, 538, 127, 536, 257, 410, 255, 408, 513, 154, 511, 152, 360, 307, 358, 305, 54, 613, 52, 611, 107, 560, 105, 558, 213, 454, 211, 452, 6, 663, 2, 659, 10, 659, 6, 655, 18, 651, 14, 647, 34, 635, 30, 631, 66, 603, 62, 599, 130, 539, 126, 535, 258, 411, 254, 407, 514, 155, 510, 151, 361, 308, 357, 304, 55, 614, 51, 610, 108, 561, 104, 557, 214, 455, 210, 451, 12, 661, 4, 653, 20, 653, 12, 645, 36, 637, 28, 629, 68, 605, 60, 597, 132, 541, 124, 533, 260, 413, 252, 405, 516, 157, 508, 149, 363, 310, 355, 302, 57, 616, 49, 608, 110, 563, 102, 555, 216, 457, 208, 449, 24, 657, 8, 641, 40, 641, 24, 625, 72, 609, 56, 593, 136, 545, 120, 529, 264, 417, 248, 401, 520, 161, 504, 145, 367, 314, 351, 298, 61, 620, 45, 604, 114, 567, 98, 551, 220, 461, 204, 445, 48, 649, 16, 617, 80, 617, 48, 585, 144, 553, 112, 521, 272, 425, 240, 393, 528, 169, 496, 137, 375, 322, 343, 290, 69, 628,

In [17]:
total_AWE = AWE1_1 + AWE_2
e_total = e1_data1 + e2_data
R_total = SEC_R + DEC_R
print("還未去除重複error data的數量:",np.size(e_total))
print("還未去除重複餘數的數量:", np.size(R_total))

## 把重複的 AWE 及 error_data 去除
AWE = remove_duplicates(total_AWE)
new_e_total = remove_duplicates(e_total)
new_R_total = remove_duplicates(R_total)
error_total_count = np.size(new_e_total)
print("已去除重複error data的數量:",error_total_count)
print("已未去除重複餘數的數量:",np.size(new_R_total))
print("已未去除重複AWE的數量:",np.size(AWE))
print("所有的AWE為:\n",AWE)
print("所有算術錯誤的資料:\n", new_e_total)

還未去除重複error data的數量: 392
還未去除重複餘數的數量: 392
已去除重複error data的數量: 342
已未去除重複餘數的數量: 342
已未去除重複AWE的數量: 342
所有的AWE為:
 [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 3, -3, 5, -5, 9, -7, 7, -9, 17, -15, 15, -17, 33, -31, 31, -33, 65, -63, 63, -65, 129, -127, 127, -129, 257, -255, 255, -257, 513, -511, 511, -513, 1025, -1023, 1023, -1025, 2049, -2047, 2047, -2049, 4097, -4095, 4095, -4097, 8193, -8191, 8191, -8193, 6, -6, 10, -10, 18, -14, 14, -18, 34, -30, 30, -34, 66, -62, 62, -66, 130, -126, 126, -130, 258, -254, 254, -258, 514, -510, 510, -514, 1026, -1022, 1022, -1026, 2050, -2046, 2046, -2050, 4098, -4094, 4094, -4098, 8194, -8190, 8190, -8194, 12, -12, 20, -20, 36, -28, 28, -36, 68, -60, 60, -68, 132, -124, 124, -132, 260, -252, 252, -260, 516, -508, 508, -516, 1028, -1020, 1020, -1028, 2052, -2044, 2044, -2052, 4100, -4092, 4092, -4100, 8196, -8188, 8188, -8196, 24, -24, 40, -40, 72, -56, 56, -

### 所有AWE數量: $4*C^{bits}_{2}$ $-$ $2*(bits-1)$ - $2*(bits-2)$
* $2*(bits-1)$: 所有的 single AWE都可以分解成已重複的 double AWE, 除了MSB例外
* EX: $(+,0)$ 只能分解成 $(0,++)$ , $(-,0)$ 只能分解成 $(0,--)$
* $2*(bits-2)$: double AWE重複的個數
* EX: $(+,0,-)$ = $(0,+,+)$ and $(-,0,+)$ = $(0,-,-)$

### Trade-off Algorithm: 2025/05/27 使用 Vivado HLX IP 驗證
* 此解出來的數字為位置及正負號 EX: $-1$ 代表 $-2^0$ , $-5$ 代表 $-2^4$ 以此類推

In [20]:
import math
import time
### 給定一個DEC_R, 找錯誤位置h1及h2(包含錯誤正負號)
import numpy as np

h1 = 0; h2 = 0; h_temp = 0
possible_h1 = []; possible_h2 = []; possible_AWE = []
output = []; possible_SE_h1=[]

start_time = time.time()  # 記錄開始時間
for R, error in zip(new_R_total, new_e_total): 
    ip.write_mm(0, R)
    
    h1 = twos_complement_32bit(ip.read_mm(4))                  
    h2 = twos_complement_32bit(ip.read_mm(8))
    
    s2 = get_sign(h2)
    s1 = get_sign(h1)
    
    possible_h1.append(h1)                  
    possible_h2.append(h2)                  
    output.append((error - (s1 * 2**(abs(h1)-1)) - (s2 * 2**(abs(h2)-1))) // AN)

            
end_time = time.time()  # 記錄結束時間
elapsed_time = end_time - start_time  # 計算執行時間
print(f"執行時間: {elapsed_time:.6f} 秒")
print("更正後的值是否和原data相同:", data1 == output.pop())
print(elapsed_time / np.size(possible_h1))
## print(output)
### print(possible_h1)
### print(possible_h2)
##  print("更正後的值:",output)
print(np.size(possible_h1))
print(np.size(possible_h2))
##print("輸入的 error data的總數是否和經過演算法後相同:", error_total_count == np.size(possible_h1))
##print(possible_SE_h1)

執行時間: 0.040277 秒
更正後的值是否和原data相同: False
0.00011776971538164462
342
342
